<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-1-Introduction-to-Generative-AI-and-Agentic-AI/M1LC1_LLM_Input_Output_and_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Input/Ouput and Prompting

This section introduces the foundational concepts of using Language Models (LLMs) and Chat Models in LangChain, focusing on how to structure inputs and extract meaningful outputs. It covers:


- **Model I/O Fundamentals**:
  - **LLMs**: Accept plain text and return plain text, useful for traditional completion tasks.
  - **Chat Models**: Take structured messages (like user/assistant roles) and return conversational outputs.
  - **Prompts**: Structured inputs designed to guide LLM behavior effectively.
  - **Output Parsers**: Used to extract structured data or formats from raw model outputs.

A simple demo uses the `ChatOpenAI` class to invoke the GPT-4o-mini model with a sample prompt and display its response, setting the stage for deeper exploration into LangChain workflows.


## Install OpenAI and LangChain dependencies

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 --quiet

## Enter API Keys & Setup Environment Variables

In [ ]:
import os
import getpass

# OpenAI API Key (for chat & embeddings)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (https://platform.openai.com/account/api-keys):\n")

# Model I/O

In LangChain, the central part of any application is the language model. This module provides crucial tools for working effectively with any language model, ensuring it integrates smoothly and communicates well.

### Key Components of Model I/O

**LLMs and Chat Models (used interchangeably):**
- **LLMs:**
  - **Definition:** Pure text completion models.
  - **Input/Output:** Receives a text string and returns a text string.
- **Chat Models:**
  - **Definition:** Based on a language model but with different input and output types.
  - **Input/Output:** Takes a list of chat messages as input and produces a chat message as output.
- **Prompts:** Helps in creating adaptable and context-sensitive prompts that direct the responses of the language model.
- **Output Parsers:** Helps in extracting and shaping information from the outputs of language models. This is valuable for turning the language model's raw output into structured data or specific formats needed


## Chat Models and LLMs

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
prompt = """Explain what is Agentic AI in 2 bullets?"""
print(prompt)

In [ ]:
response = chatgpt.invoke(prompt)
response

In [ ]:
print(response.content)

In [ ]:
from IPython.display import display, Markdown

display(Markdown(response.content))

In [ ]:
chatgpt = ChatOpenAI(model_name="gpt-5-mini",
                     reasoning = {
                        "effort": "medium",  # 'low', 'medium', or 'high'
                        "summary": "detailed",  # 'detailed', 'auto', or None
                     },
                     temperature=0)

In [ ]:
response = chatgpt.invoke(prompt)
response

In [ ]:
display(Markdown(response.additional_kwargs['reasoning']['summary'][0]['text']))

In [ ]:
display(Markdown(response.content[0]['text']))

## Message Types

ChatModels process a list of messages, receiving them as input and responding with a message. Messages are characterized by a few distinct types and properties:

- **Role:** Indicates who is speaking in the message. LangChain offers different message classes for various roles.
- **Content:** The substance of the message, which can vary:
  - A string (commonly handled by most models)
  - A list of dictionaries (for multi-modal inputs, where each dictionary details the type and location of the input)

Additionally, messages have an `additional_kwargs` property, used for passing extra information specific to the message provider, not typically general. A well-known example is `function_call` from OpenAI.

### Specific Message Types

- **HumanMessage:** A user-generated message, usually containing only content.
- **AIMessage:** A message from the model, potentially including `additional_kwargs`, like `tool_calls` for invoking OpenAI tools.
- **SystemMessage:** A message from the system instructing model behavior, typically containing only content. Not all models support this type.


### Simple Conversational Prompting

In [ ]:
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

prompt = """Can you explain what is P/E Ratio in 2 lines"""

messages = [
    SystemMessage(content="Act as a helpful assistant who simplifies things with easy to understand examples."),
    HumanMessage(content=prompt),
]

messages

In [ ]:
response = chatgpt.invoke(messages)
response

In [ ]:
display(Markdown(response.content.replace('$', '\\$')))

In [ ]:
messages

In [ ]:
messages.append(response)

prompt = """What did we discuss so far?"""
new_message = ('user', prompt) # OR HumanMessage(content=prompt)
messages.append(new_message)
messages

In [ ]:
response = chatgpt.invoke(messages)
response.content

## Prompt Templates
Prompt templates are pre-designed formats used to generate prompts for language models. These templates can include instructions, few-shot examples, and specific contexts and questions suited for particular tasks.

LangChain provides tools for creating and using prompt templates. It aims to develop model-agnostic templates to facilitate the reuse of existing templates across different language models. Typically, these models expect prompts in the form of either a string or a list of chat messages.

### Types of Prompt Templates

- **PromptTemplate:**
  - Used for creating string-based prompts.
  - Utilizes Python's `str.format` syntax for templating, supporting any number of variables, including scenarios with no variables.

- **ChatPromptTemplate:**
  - Designed for chat models, where the prompt consists of a single or list of chat messages.
  - Each chat message includes content and a role parameter. For instance, in the OpenAI Chat Completions API, a chat message could be assigned to an AI assistant, a human, or a system role.



##### ChatPromptTemplate with simple prompts

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# more complex prompt with placeholders
prompt = """Explain to me briefly about {topic} in 1 line."""

chat_template = ChatPromptTemplate.from_template(prompt)
chat_template

Chains enable you to input data dynamically at runtime

In [ ]:
simple_chain = (chat_template
                    |
                 chatgpt)
simple_chain

In [ ]:
topics = ['Generative AI', 'Machine Learning', 'Deep Learning']
responses = simple_chain.map().invoke(topics)
for response in responses:
  print(response.content)
  print('-----')

##### ChatPromptTemplate with sequence of prompts

In [ ]:
messages = [
        ("system", "Act as an expert in insurance and provide brief answers"),
        ("human", "what is your name?"),
        ("ai", "my name is AIBot"),
        ("human", "{user_prompt}"),
]
chat_template = ChatPromptTemplate.from_messages(messages)
chat_template

In [ ]:
simple_chain = chat_template | chatgpt

In [ ]:
text_prompts = ["what is your name?",
                "explain healthcare insurance to me"]


In [ ]:
responses = simple_chain.map().invoke(text_prompts)
for response in responses:
  print(response.content)
  print('-----')

## Structured Prompting

Output parsers are essential in Langchain for structuring the responses from language models. Below, we will look at how to prompt LLMs to generate structured outouts.

![](https://i.imgur.com/qtXFjf3.png)

- **Pydantic Schema:**
  - This schema allows the specification of an arbitrary Pydantic Model to force LLMs for outputs matching that schema. Pydantic's BaseModel functions similarly to a Python dataclass but includes type checking and coercion.

- **Structured Outputs:**
  - LangChain provides a method, `with_structured_output()`, that automates the process of binding the schema to the model and parsing the output. This helper function is available for all model providers that support structured output.



In [ ]:
from typing import List
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Define your desired data structure - like a python data class.
class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description="The original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description="Response to the customer in their original language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")

In [ ]:
# And a query intented to prompt a language model to populate the data structure.
prompt_txt = """
             Act as an Information Technology (IT) customer support agent. For the IT support message mentioned below
             in triple backticks use the given format when generating the output response

             Customer IT support message:
             ```{it_support_msg}```
             """


prompt = PromptTemplate.from_template(template=prompt_txt)
prompt.pretty_print()

In [ ]:
structured_llm = chatgpt.with_structured_output(ITSupportResponse)
structured_llm

In [ ]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

formatted_msgs = [{"it_support_msg": msg}
                    for msg in it_support_queue]
formatted_msgs[0]

In [ ]:
qa_chain = prompt | structured_llm

In [ ]:
responses = qa_chain.map().invoke(formatted_msgs)

In [ ]:
responses[0]

In [ ]:
responses[0].model_dump()

In [ ]:
responses = [response.model_dump() for response in responses]

In [ ]:
import pandas as pd

df = pd.DataFrame(responses)
df

## Streaming in LLMs

All language model interfaces (LLMs) in LangChain implement the `Runnable` interface, which provides default methods such as `ainvoke`, `batch`, `abatch`, `stream`, and `astream`. This setup equips all LLMs with basic streaming capabilities.

### Streaming Defaults:

- **Synchronous Streaming:** By default, streaming operations return an `Iterator` that yields a single value, the final result from the LLM provider.
- **Asynchronous Streaming:** Similarly, async streaming defaults to returning an `AsyncIterator` with the final result.

### Limitations:

- These default implementations do not support token-by-token streaming. For such detailed streaming, the LLM provider must offer native support. However, the default setup ensures that your code expecting an iterator of tokens will function correctly within these constraints.


In [ ]:
prompt = """Explain to me what is utlization review in 3 bullets"""

for chunk in chatgpt.stream(prompt):
    print(chunk.content, end='')